<a href="https://colab.research.google.com/github/egabrielvice/DATA602_ipynb/blob/7/assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 7**
# **Course**: DATA 602

# **Name**: Escarlet Gabriel Vicente
# **Weeks 8 & 9 - Pandas**


#Introduction


For this assignment, an eCommerce transaction dataset from Kaggle was selected.

**Dataset source**: https://www.kaggle.com/datasets/carrie1/ecommerce-data

This dataset contains transactional records from an online retail store, including invoice numbers, product identifiers, quantities, unit prices, transaction dates, customer identifiers, and country information.

The dataset was chosen because it represents real-world business data and contains multiple data quality issues, such as missing values, duplicate records, inconsistent data types, invalid values, and character encoding challenges. These characteristics make it well suited for demonstrating data exploration and data wrangling using Pandas.

______________
# Data Exploration
In order to understand the structure, content, and overall quality of the dataset before applying any transformations, an initial data exploration is conducted.

### 1. Load Libraries
The analysis begins by loading the required Python libraries. Pandas serves as the
primary tool for data manipulation, while NumPy supports numerical operations.
Visualization libraries are included to allow graphical exploration when appropriate.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 2. Load Data
The dataset is imported into a Pandas DataFrame to enable efficient exploration and analysis. A predefined character encoding is applied during ingestion to resolve encoding discrepancies and ensure accurate data processing.

An initial preview of the dataset is displayed to gain a general understanding of the variables, data formats, and overall structure. This step provides early insight into column names, value ranges, and potential inconsistencies before deeper inspection.

In [ ]:
df = pd.read_csv("/data.csv", encoding="latin1")
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


### 3. Structure & Info

The structure of the dataset is examined using the `info()` method, which provides information on the number of observations, variable names, data types, and non-null counts. This step is essential for identifying improperly coded variables, such as dates stored as strings or numeric identifiers stored as floating-point values.

Additionally, non-null counts reveal the presence and extent of missing values across columns, informing later decisions regarding data cleaning and imputation.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49912 entries, 0 to 49911
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    49912 non-null  object 
 1   StockCode    49912 non-null  object 
 2   Description  49770 non-null  object 
 3   Quantity     49912 non-null  int64  
 4   InvoiceDate  49912 non-null  object 
 5   UnitPrice    49911 non-null  float64
 6   CustomerID   32030 non-null  float64
 7   Country      49911 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 3.0+ MB


### 4. Summary Statistics

Summary statistics are computed to examine the central tendency and dispersion of numeric variables. Measures such as the mean, median, minimum, maximum, and quartiles provide insight into typical values, variability, and the potential presence of outliers.

This step helps assess whether numeric values fall within reasonable ranges and highlights anomalies that may require further investigation.

In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,49912.000000,49911.000000,32030.000000
mean,8.294218,6.423712,15426.016422
std,55.576019,166.653120,1752.300279
min,-9360.000000,0.000000,12347.000000
25%,1.000000,1.250000,14057.000000
50%,2.000000,2.510000,15464.000000
75%,8.000000,4.250000,17041.000000
max,2880.000000,16888.020000,18283.000000


### 5. Missing Values

A dedicated check for missing values is performed to quantify the number of null entries in each column. Identifying missing data is essential, as unaddressed null values can bias results or cause errors in calculations and aggregations.

Understanding where and how frequently missing values occur informs the choice of appropriate handling strategies, such as imputation, labeling, or row removal.

In [ ]:
df.isna().sum().sort_values(ascending=False)

,0
CustomerID,17882
Description,142
UnitPrice,1
Country,1
Quantity,0
StockCode,0
InvoiceNo,0
InvoiceDate,0


### 6. Duplicated Rows

Duplicate rows are identified to determine whether repeated records exist in the dataset. Duplicate entries can distort summary statistics and aggregated results if not handled appropriately.

Detecting duplicates at this stage ensures that subsequent analyses are based on unique and accurate observations.

In [ ]:
df.duplicated().sum()

np.int64(537)

# Data Exploration Findings

The exploratory analysis reveals several data quality issues that must be addressed before reliable analysis can be conducted:


*   Missing values are present in customer identifiers and product descriptions.
*   Some variables, such as invoice dates, are stored as strings instead of datetime objects.
*   The dataset contains duplicate records.
*   Invalid values, including negative quantities, are present.
*   The dataset requires special handling due to non-UTF-8 character encoding.

These findings confirm that systematic data wrangling is necessary prior to further analysis.

# Data Wrangling

In order to prepare the dataset for meaningful analysis, a series of data wrangling
steps are applied. These steps address data quality issues identified during the
exploration phase, including inconsistent formatting, missing values, invalid entries,
and redundant information. Each subsection corresponds to a specific transformation
performed on the dataset.

### 1. Modify Multiple Column Names

Column names are standardized by converting them to lowercase and replacing spaces
with underscores. This improves readability, ensures consistency, and aligns with
common Python and Pandas naming conventions.

In [ ]:
df = df.rename(columns={
    "InvoiceNo": "invoice_no",
    "StockCode": "stock_code",
    "Description": "description",
    "Quantity": "quantity",
    "InvoiceDate": "invoice_date",
    "UnitPrice": "unit_price",
    "CustomerID": "customer_id",
    "Country": "country"
})
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### 2. Review and Correct Data Types

The structure of the dataset is examined to identify variables that are improperly
coded. In particular, date variables stored as strings are converted to datetime
objects to enable accurate time-based analysis.

In [ ]:
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49912 entries, 0 to 49911
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   invoice_no    49912 non-null  object        
 1   stock_code    49912 non-null  object        
 2   description   49770 non-null  object        
 3   quantity      49912 non-null  int64         
 4   invoice_date  49912 non-null  datetime64[ns]
 5   unit_price    49911 non-null  float64       
 6   customer_id   32030 non-null  float64       
 7   country       49911 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 3.0+ MB


### 3. Fix Missing and Invalid Values

Missing values are present in several columns, most notably in customer identifiers.
These missing values likely represent anonymous or guest transactions and are handled
in a way that preserves the transaction records while clearly distinguishing missing
identifiers.

In [ ]:
df["customer_id"].isna().sum()

np.int64(17882)

In [ ]:
df["customer_id"] = df["customer_id"].fillna(-1)
df["customer_id"].isna().sum()

np.int64(0)

### 4. Create New Columns from Existing Data

A new column representing the total transaction value is created by multiplying the
quantity of items purchased by the unit price. This derived variable provides a more
meaningful measure for financial analysis.

In [ ]:
df.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

In [ ]:
df["total_price"] = df["quantity"] * df["unit_price"]
df.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country', 'total_price'],
      dtype='object')

### 5. Drop Unnecessary Columns

Columns that are not required for the analytical objectives are removed to reduce
noise and simplify the dataset.

In [ ]:
df = df.drop(columns=["description"])
df.columns

Index(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price',
       'customer_id', 'country', 'total_price'],
      dtype='object')

### 6. Drop Invalid Rows

Rows containing invalid values are removed from the dataset. Specifically,
transactions with non-positive quantities are excluded, as these values represent
returns or data entry errors that could distort summary statistics.

In [ ]:
df.shape

(49912, 8)

In [ ]:
df = df[df["quantity"] > 0]
df.shape

(48904, 8)

### 7. Sort the Dataset by Multiple Variables

The dataset is sorted using multiple variables to improve interpretability.
Sorting by country and transaction value allows for easier identification of
high-value transactions within each group.

In [ ]:
df.sort_values(by=["country", "total_price"], ascending=[True, False]).head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country,total_price
45541,540267,22720,240,2011-01-06 11:12:00,4.25,12415.0,Australia,1020.0
45542,540267,22722,168,2011-01-06 11:12:00,3.45,12415.0,Australia,579.6
45536,540267,22492,576,2011-01-06 11:12:00,0.55,12415.0,Australia,316.8
45537,540267,22619,80,2011-01-06 11:12:00,3.39,12415.0,Australia,271.2
45534,540267,21915,240,2011-01-06 11:12:00,1.06,12415.0,Australia,254.4


### 8. Filter the Dataset Based on a Condition

A subset of the dataset is created by filtering transactions that exceed a specified
transaction value threshold. This enables focused analysis of higher-value purchases.

In [ ]:
high_value = df[df["total_price"] > 500]
high_value.head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country,total_price
178,536387,79321,192,2010-12-01 09:58:00,3.82,16029.0,United Kingdom,733.44
179,536387,22780,192,2010-12-01 09:58:00,3.37,16029.0,United Kingdom,647.04
180,536387,22779,192,2010-12-01 09:58:00,3.37,16029.0,United Kingdom,647.04
181,536387,22466,432,2010-12-01 09:58:00,1.45,16029.0,United Kingdom,626.40
182,536387,21731,432,2010-12-01 09:58:00,1.25,16029.0,United Kingdom,540.00


In [ ]:
high_value.shape

(132, 8)

### 9. Standardize String Values

Text values in the country column are converted to a consistent case to prevent
grouping inconsistencies caused by variations in capitalization.

In [ ]:
df["country"] = df["country"].str.upper()
df["country"].value_counts().head()

,count
country,
UNITED KINGDOM,46075
GERMANY,741
FRANCE,600
EIRE,374
SPAIN,173


### 10. Validate Numeric Values in a Column

A validation check is performed to determine whether invoice numbers contain only
numeric characters. This helps identify mixed or improperly formatted identifiers.

In [ ]:
df["invoice_no"].str.isnumeric().value_counts()

,count
invoice_no,
True,48904


### 11. Group the Dataset by One Variable

The dataset is grouped by country, and summary statistics including the mean,
minimum, and maximum transaction values are calculated. This provides insight into
how transaction values vary across regions.

In [ ]:
df.groupby("country")["total_price"].agg(["mean", "min", "max"]).head()

,mean,min,max
country,,,
AUSTRALIA,102.340375,10.20,1020.00
AUSTRIA,34.650000,10.20,120.00
BAHRAIN,205.740000,205.74,205.74
BELGIUM,19.051684,2.50,54.00
CHANNEL ISLANDS,21.384118,14.85,50.00


### 12. Group the Dataset by Two Variables and Sort Results

The dataset is grouped by both country and product code, and the aggregated results
are sorted to highlight the most significant combinations across multiple dimensions.


In [ ]:
df.groupby(["country", "stock_code"])["total_price"] \
  .mean() \
  .sort_values(ascending=False) \
  .head(10)

country         stock_code
UNITED KINGDOM  AMAZONFEE     13541.330000
JAPAN           22328          3794.400000
UNITED KINGDOM  22833          1201.815000
SWEDEN          85232B         1188.000000
AUSTRALIA       22720          1020.000000
                22722           579.600000
UNITED KINGDOM  22275           527.850000
                22830           518.058000
NETHERLANDS     22326           489.600000
UNITED KINGDOM  DOT             429.354194
Name: total_price, dtype: float64

# Conclusions  

This analysis demonstrated the importance of systematic data exploration and wrangling when working with real-world transactional data. Several key insights emerged from the cleaned dataset:

* The dataset contained substantial data quality issues, including missing customer identifiers, improperly formatted date variables, duplicate records, and invalid quantity values. Addressing these issues was necessary to ensure accurate analysis.
* Creating a derived transaction-level metric (`total_price`) enabled more meaningful financial analysis and supported effective filtering, sorting, and aggregation.
* Transaction values varied significantly across countries and products, with a small subset of high-value transactions accounting for a disproportionate share of total revenue.
* Grouping by multiple dimensions (country and product) revealed that certain product–country combinations consistently generate higher transaction values.

Overall, this assignment highlights how uncleaned data can distort results and how Pandas provides efficient tools to transform raw data into an analysis-ready format.

With additional time, further analysis could include:

* Visualizations to explore sales trends over time
* Deeper investigation into customer purchasing behavior
* Integration of external datasets to enrich geographic or product-level insights
* Time-series or predictive modeling to forecast future sales patterns